# Tasks

Tasks are a basic execution unit in Airflow. This page focuses on details related to working with tasks.

## Communication

In Airflow, setting up communication between tasks in a DAG is crucial. Airflow provides a special mechanism for this called XCom (short for 'Cross-Communication'). XCom is a system for passing small amounts of data between tasks by serializing Python objects and storing them in the metadata database. This allows tasks to share information and enables Airflow to track inter-task communication.

Check more:

- [XComs](https://airflow.apache.org/docs/apache-airflow/stable/core-concepts/xcoms.html) page.
- [Passing arbitrary objects as argumemnts](https://airflow.apache.org/docs/apache-airflow/stable/core-concepts/taskflow.html#passing-arbitrary-objects-as-arguments) section of the TaskFlow description.

---

As an example, consider passing arguments in the modern Airflow syntax - task flow. You just have to work with tasks just like with python functions, all stuff associated related to XCom is implemented by the `task` decorator.

The following cell takes the string from `echo` and saves it to the disk the `save` task.

In [1]:
%%writefile /opt/airflow/dags/tutorial.py
from pathlib import Path
from airflow.decorators import dag, task
import pendulum

now = pendulum.now("UTC")

@dag("etl")
def etl(start_date=now, schedule="@daily", catchup=False):
    @task
    def echo() -> str:
        return "message"

    @task
    def save(temps: str) -> None:
        Path("/tmp/airflow_data").write_text(temps)

    data = echo()
    save(data) 

etl()

Overwriting /opt/airflow/dags/tutorial.py


The following cell runs dag and displays the contents of the `/tmp/airflow_data` cell.

In [5]:
%%bash
airflow db migrate &> /dev/null
airflow tasks clear -s 2022-01-01 -e 2022-01-01 -y etl
airflow dags backfill -s 2022-01-01 -e 2022-01-01 etl

[2025-03-23T13:00:57.099+0000] {dagbag.py:588} INFO - Filling up the DagBag from /opt/airflow/dags
[2025-03-23T13:00:59.928+0000] {dagbag.py:588} INFO - Filling up the DagBag from /opt/airflow/dags
[2025-03-23T13:01:00.220+0000] {executor_loader.py:258} INFO - Loaded executor: SequentialExecutor
[2025-03-23T13:01:00.287+0000] {base_executor.py:169} INFO - Adding to queue: ['airflow', 'tasks', 'run', 'etl', 'echo', 'backfill__2022-01-01T00:00:00+00:00', '--depends-on-past', 'ignore', '--local', '--pool', 'default_pool', '--subdir', 'DAGS_FOLDER/tutorial.py', '--cfg-path', '/tmp/tmp99vnbjxg']
[2025-03-23T13:01:00.293+0000] {sequential_executor.py:84} INFO - Executing command: ['airflow', 'tasks', 'run', 'etl', 'echo', 'backfill__2022-01-01T00:00:00+00:00', '--depends-on-past', 'ignore', '--local', '--pool', 'default_pool', '--subdir', 'DAGS_FOLDER/tutorial.py', '--cfg-path', '/tmp/tmp99vnbjxg']
[2025-03-23T13:01:02.332+0000] {dagbag.py:588} INFO - Filling up the DagBag from /opt/airflow/

In [6]:
!cat /tmp/airflow_data

message

As a result, the exact string generated by the `echo` task was stored.